This script will generate a tile index based on a small set of input parameters. 
The tile index can be used with cluster.py by adding a new augmentation type to "load_index"

In [1]:
import urllib, json, os

zoom = 11 #choose based on size of input geometries. should be smaller by roughly 1/2 an order of magnitude
source = 'tl_2014_census_tracts'
account = 'andrew'
id_column = 'geoid'
index_name = 'census'
sql = "WITH c AS ( \
        SELECT CDB_XYZ_Extent(x, y, "+str(zoom)+") g, x, y \
        FROM GENERATE_SERIES(0,pow(2, "+str(zoom)+")::int-1) x, GENERATE_SERIES(0,pow(2, "+str(zoom)+")::int - 1) y), \
       geoms AS ( \
        SELECT "+id_column+" as id, c.x, c.y, "+str(zoom)+" as zoom \
        FROM "+source+", c WHERE ST_Intersects("+source+".the_geom_webmercator, c.g)) \
       SELECT x, y, ARRAY_AGG(id) ids FROM geoms GROUP BY x,y ORDER BY x asc, y desc "
api_key = False


In [2]:
# Get SQL API key
api_key = os.environ.get('ANDREW_API_KEY')

In [3]:
if api_key:
    url = "http://"+account+".cartodb.com/api/v2/sql?"+urllib.urlencode({'q': sql, 'api_key': api_key})
else:
    url = "http://"+account+".cartodb.com/api/v2/sql?"+urllib.urlencode({'q': sql})
response = urllib.urlopen(url)
tiles = json.loads(response.read())
print len(tiles['rows'])

61658


In [ ]:
# reformat for lookup
output = {}
ids = []
for row in tiles['rows']:
    # x = 2**(zoom) - 1 - row['x']
    x = row['x']
    if x not in output:
        output[x] = {}
    if row['y'] not in output:
        output[x][row['y']] = []
    output[x][row['y']] = output[x][row['y']] + row['ids']
    
    # store unique geo_ids
    all_ids = set(ids)
    new_ids = set(row['ids'])
    additions = new_ids - all_ids
    ids = ids + list(additions)
print len(ids)

In [6]:
# write to disk
with open('data/'+index_name+'.11.json', 'w') as outfile:
    json.dump({"zoom": zoom, "index_name": index_name, "index": output, "geometry_directory": 'data/'+index_name }, outfile)

NameError: name 'output' is not defined

In [25]:
# divide up our records into groups of 100 for downloading
upper_aggs = {}
if not os.path.exists('data/'+index_name):
    os.makedirs('data/'+index_name)
for chunk in [ids[i:i + 100] for i in range(0, len(ids), 100)]:
    # at the same time, we'll create our aggregate augmentations for state and county
    # TODO: add zcta column to the source data so include in upper_aggs
    id_list = []
    for c in chunk:
        id_list.append("'"+str(c)+"'")
    sql = "SELECT ST_SimplifyPreserveTopology(the_geom,0.001) the_geom, "+id_column+" AS id, statefp as statefp, countyfp as countyfp FROM "+source+" WHERE "+id_column+" IN ("+','.join(id_list)+")"
    if api_key:
        url = "http://"+account+".cartodb.com/api/v2/sql?"+urllib.urlencode({'q': sql, 'format':'geojson', 'api_key': api_key})
    else:
        url = "http://"+account+".cartodb.com/api/v2/sql?"+urllib.urlencode({'q': sql, 'format':'geojson'})
    response = urllib.urlopen(url)
    try:
        data = json.loads(response.read())
        for i in data['features']:
            name = i['properties']['id']
            geom = i['geometry']
            # Format of the aggs index for use in the augmentation
            upper_aggs[name] = {'statefp': i['properties']['statefp'], 'countyfp': i['properties']['countyfp']}
            # write to disk
            with open('data/'+index_name+'/'+str(name)+'.json', 'w') as outfile:
                json.dump(geom, outfile)
    except:
        print 'Error:',response.read()


In [26]:

# Aggregation index in format:
# {census_id: {statefp: text, countyfp: text}}
with open('data/'+index_name+'_aggregates.json', 'w') as outfile:
    json.dump(upper_aggs, outfile)
        

512